<a href="https://colab.research.google.com/github/Satyendersingh024/RAG-QA-Bot/blob/main/RAG_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Retrieval-Augmented Generation (RAG) Model for QA Bot**

In [1]:
# Install the required packages (Pinecone, Cohere, and others)
!pip install pinecone-client cohere pandas scikit-learn

# Import necessary libraries
import pinecone
import cohere
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import zipfile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Step 2: Unzip the archive.zip
import zipfile
zip_path = '/content/drive/MyDrive/archive.zip'  # Replace with your actual file path
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/extracted')


In [4]:
# Step 3: List the contents of the extracted folder
!ls /content/drive/MyDrive/extracted

credits.csv   links.csv        movies_metadata.csv  ratings_small.csv
keywords.csv  links_small.csv  ratings.csv


In [5]:
# Load the CSV files into DataFrames
credits_df = pd.read_csv('/content/drive/MyDrive/extracted/credits.csv')
links_df = pd.read_csv('/content/drive/MyDrive/extracted/links.csv')
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
ratings_small_df = pd.read_csv('/content/drive/MyDrive/extracted/ratings_small.csv')
keywords_df = pd.read_csv('/content/drive/MyDrive/extracted/keywords.csv')
links_small_df = pd.read_csv('/content/drive/MyDrive/extracted/links_small.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/extracted/ratings.csv')


<ipython-input-5-f57cf595e65d>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')


In [6]:
# Initialize Pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6')


In [7]:
# Check if the index exists and create it if it doesn't
index_name = 'sample-movies'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Use the dimension specified for your embeddings
        metric='cosine',  # Use cosine similarity or another appropriate metric
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


In [8]:
# Get the index
index = pc.Index(index_name)


In [ ]:
# Initialize Cohere
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

In [9]:
# Prepare the data
# Fill missing values
movies_metadata_df['overview'] = movies_metadata_df['overview'].fillna('')

In [10]:
# Example function to get text embeddings
def get_embeddings(texts):
    response = cohere_client.embed(texts=texts)
    return response.embeddings

In [11]:
import cohere
from tqdm import tqdm
import pandas as pd
import pickle

# Initialize Cohere client
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

def get_embeddings_batch(texts, batch_size=10):
    embeddings = []
    num_batches = (len(texts) + batch_size - 1) // batch_size  # Calculate the number of batches

    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings", total=num_batches):
        batch = texts[i:i + batch_size]
        try:
            response = cohere_client.embed(texts=batch)
            embeddings.extend(response.embeddings)
        except Exception as e:
            print(f"Error generating embeddings for batch starting at index {i}: {e}")
    return embeddings

def save_embeddings(embeddings, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(embeddings, file)

# Load the CSV file
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
texts = movies_metadata_df['overview'].fillna('').tolist()

# Process embeddings in smaller batches
batch_size = 10  # Smaller batch size to reduce API load
embeddings = get_embeddings_batch(texts, batch_size)

# Save the embeddings to a file
save_embeddings(embeddings, '/content/drive/MyDrive/embeddings.pkl')

print("Embeddings generation and saving completed.")


<ipython-input-11-6e3b9201dad5>:27: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
Generating embeddings:   0%|          | 3/4547 [00:00<06:49, 11.11it/s]

Error generating embeddings for batch starting at index 0: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 10: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 20: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API ca

Generating embeddings:   0%|          | 5/4547 [00:00<06:26, 11.76it/s]

Error generating embeddings for batch starting at index 30: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 40: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 50: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API c

Generating embeddings:   0%|          | 9/4547 [00:00<05:55, 12.76it/s]

...

Generating embeddings: 100%|█████████▉| 4543/4547 [05:24<00:00, 13.12it/s]

Error generating embeddings for batch starting at index 45410: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45420: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45430: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1

Generating embeddings: 100%|██████████| 4547/4547 [05:25<00:00, 13.99it/s]

Error generating embeddings for batch starting at index 45440: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45450: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45460: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1

Embeddings generation and saving completed.


In [12]:
import pinecone
import pickle

# Correct API key and environment (verify these from Pinecone dashboard)
api_key = 'bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6'
index_name = 'sample-movies'
index_host = 'https://sample-movies-blulkao.svc.aped-4627-b74a.pinecone.io'

# Initialize Pinecone
pc = pinecone.Pinecone(api_key=api_key)

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to the index using the correct host
index = pinecone.Index(index_name, host=index_host)

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Prepare data for upsert (movie IDs and embeddings)
movie_ids = movies_metadata_df['id'].tolist()
vectors = [(str(movie_id), embedding) for movie_id, embedding in zip(movie_ids, embeddings)]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(vectors[i : i + batch_size])
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.core.client.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
        print(e.body)
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

Upserting to Pinecone completed.


In [ ]:
import pinecone
import cohere
import pickle
import pandas as pd
import time

# Initialize Pinecone and Cohere
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6', environment='us-east-1')
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Define Pinecone index parameters
index_name = 'sample-movies'

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to Pinecone index
index = pc.Index(index_name)

def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False  # Set to True if you want to include vector values
    )
    return results

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    max_retries = 5
    retry_delay = 10  # seconds
    for attempt in range(max_retries):
        try:
            response = cohere_client.generate(
                model='command-xlarge-20220901',  # Choose the appropriate model
                prompt=prompt,
                max_tokens=100
            )
            return response.generations[0].text
        except cohere.TooManyRequestsError as e:
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        except Exception as e:
            print(f"An error occurred while generating a response: {e}")
            break

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata with dtype option to avoid warnings
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)

movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert (movie IDs and embeddings)
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(
            vectors=vectors[i:i + batch_size],
            namespace="default"  # Use default namespace, or specify if using multiple namespaces
        )
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

# Example Query
query_text = "Tell me about movies with a strong female lead and action-packed scenes."
try:
    query_embedding = cohere_client.embed(texts=[query_text]).embeddings[0]
except cohere.TooManyRequestsError as e:
    print(f"Rate limit exceeded while generating embedding: {e}")
    # Implement additional handling if necessary
    query_embedding = None

if query_embedding:
    # Query Pinecone
    top_results = query_pinecone(query_embedding)
    retrieved_ids = [match.id for match in top_results.matches]

    # Retrieve the corresponding movie overviews
    retrieved_texts = movies_metadata_df[movies_metadata_df['id'].isin(retrieved_ids)]['overview'].tolist()

    # Prepare prompt for generative model
    prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

    # Generate response
    response = generate_response(prompt)
    print("Generated Response:", response)


Upserting to Pinecone completed.
Rate limit exceeded while generating embedding: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


In [ ]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
import pinecone
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer

# Initialize Pinecone
api_key = 'bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6'
index_name = 'sample-movies'
index_host = 'https://sample-movies-blulkao.svc.aped-4627-b74a.pinecone.io'

# Initialize Pinecone client
pc = pinecone.Pinecone(api_key=api_key, environment='us-east-1') # Use Pinecone class to initialize client
index = pc.Index(index_name)

# Initialize SentenceTransformer for local embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)
movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(vectors=vectors[i:i + batch_size], namespace="default")
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")


Upserting to Pinecone completed.


In [ ]:
import pickle
import pandas as pd

# Load saved embeddings and check content
try:
    with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
        embeddings = pickle.load(file)
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    embeddings = []  # Set embeddings to an empty list if file not found

# Verify the type and dimensions of embeddings
print(f"Type of embeddings: {type(embeddings)}")

# Check if embeddings are loaded properly
if len(embeddings) == 0:
    print("Embeddings list is empty.")
else:
    print(f"Sample embedding: {embeddings[0]}")
    print(f"Number of embeddings: {len(embeddings)}")

# Load movie metadata
try:
    movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)
    print("Movie metadata columns:", movies_metadata_df.columns)
    print("Sample data:", movies_metadata_df.head())
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    movies_metadata_df = pd.DataFrame()  # Set to empty DataFrame if file not found

# Ensure movie_ids and embeddings have the same length
if len(movies_metadata_df) == 0 or len(embeddings) == 0:
    print("Movie metadata or embeddings are not loaded properly.")
else:
    movie_ids = movies_metadata_df['id'].tolist()

    if len(movie_ids) != len(embeddings):
        raise ValueError("The length of movie IDs and embeddings do not match.")

    # Prepare data for upsert
    vectors = [
        {
            "id": str(movie_id),
            "values": list(map(float, embedding)),  # Ensure embeddings are floats
            "metadata": {"id": str(movie_id)}
        } for movie_id, embedding in zip(movie_ids, embeddings)
    ]

    # Upsert embeddings to Pinecone in smaller batches with error handling
    batch_size = 10
    for i in range(0, len(vectors), batch_size):
        try:
            batch_vectors = vectors[i:i + batch_size]
            index.upsert(vectors=batch_vectors, namespace="default")
            print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
        except pinecone.exceptions.ForbiddenException as e:
            print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
        except Exception as e:
            print(f"An error occurred for batch starting at index {i}: {e}")

    print("Upserting to Pinecone completed.")


Type of embeddings: <class 'list'>
Embeddings list is empty.
Movie metadata columns: Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Sample data:    adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [

In [ ]:
import cohere
import pickle
import pandas as pd

# Initialize Cohere client
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)

# Prepare texts for embedding
texts = movies_metadata_df['overview'].tolist()  # Assuming you want to embed the 'overview' column

# Generate embeddings
try:
    response = cohere_client.embed(texts=texts)
    embeddings = response.embeddings
    print(f"Generated {len(embeddings)} embeddings.")
except Exception as e:
    print(f"Error generating embeddings: {e}")
    embeddings = []

# Save embeddings to a pickle file
with open('/content/drive/MyDrive/embeddings.pkl', 'wb') as file:
    pickle.dump(embeddings, file)


Error generating embeddings: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


In [ ]:
import pinecone

# Initialize Pinecone
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6')

# Define Pinecone index parameters
index_name = 'sample-movies'
index_host = 'https://sample-movies-blulkao.svc.aped-4627-b74a.pinecone.io'

# Connect to Pinecone index
index = pinecone.Index(index_name, host=index_host)

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Prepare data for upsert (movie IDs and embeddings)
movie_ids = movies_metadata_df['id'].tolist()
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,  # Ensure embeddings are in the correct format
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        batch_vectors = vectors[i:i + batch_size]
        index.upsert(vectors=batch_vectors, namespace="default")
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")


Upserting to Pinecone completed.


In [ ]:
import cohere
import pandas as pd

# Initialize Cohere client
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)

# Define query function for Pinecone
def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False,  # Set to True if you want to include vector values
        namespace="default"
    )
    return results

# Generate query embedding
query_text = "Tell me about movies with a strong female lead and action-packed scenes."
try:
    response = cohere_client.embed(texts=[query_text])
    query_embedding = response.embeddings[0]
    print("Generated query embedding.")
except Exception as e:
    print(f"Error generating query embedding: {e}")
    query_embedding = []

# Query Pinecone
try:
    top_results = query_pinecone(query_embedding)
    retrieved_ids = [match.id for match in top_results.matches]
    print(f"Retrieved IDs: {retrieved_ids}")
except Exception as e:
    print(f"Error querying Pinecone: {e}")
    retrieved_ids = []

# Retrieve the corresponding movie overviews
if retrieved_ids:
    retrieved_texts = movies_metadata_df[movies_metadata_df['id'].isin(retrieved_ids)]['overview'].tolist()
    print("Retrieved movie overviews.")
else:
    retrieved_texts = []

# Prepare prompt for generative model
prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

# Generate response
def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    try:
        response = cohere_client.generate(
            model='large',  # Choose the appropriate model
            prompt=prompt,
            max_tokens=100
        )
        return response.generations[0].text
    except Exception as e:
        print(f"Error generating response: {e}")
        return ""

response = generate_response(prompt)
print("Generated Response:", response)


Error generating query embedding: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error querying Pinecone: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 15 Sep 2024 16:10:06 GMT', 'Content-Type': 'text/plain', 'Content-Length': '9', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Wrong API key', 'www-authenticate': 'Wrong API key', 'server': 'envoy'})
HTTP response body: Forbidden

Error generating response: status_code: 400, body: {'message': "invalid request: model 'packed-embed-english-v2.0' is not supported by the generate API"}
Generated Response: 


In [ ]:
def generate_response(prompt):
    """
    Generate a response using a local model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    response = model.encode([prompt])
    return response

# Prepare prompt for generating response
prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

# Generate response
response = generate_response(prompt)
print("Generated Response:", response)


Generated Response: [[ 2.52061095e-02 -2.16245782e-02 -6.67258799e-02 -2.67860107e-02
  -2.04798803e-02  1.28051564e-01  1.62249040e-02 -4.83349897e-02
  -1.13886287e-02 -5.46882376e-02 -1.65482480e-02  5.58066228e-03
  -3.96384075e-02  2.27480195e-02  2.47919913e-02  1.10172406e-02
   7.79798776e-02 -7.22616836e-02  4.46233600e-02  4.77510989e-02
   4.76576947e-02  1.18645523e-02  3.80065441e-02  4.26585488e-02
  -9.59874317e-02  8.25551746e-04  9.22793821e-02  9.81113613e-02
  -1.21091172e-01 -1.14398897e-02  4.11729999e-02  1.49853006e-01
  -3.01929284e-02 -2.44478248e-02 -3.40669155e-02  8.32059700e-03
  -1.61290057e-02  4.63753529e-02 -3.26164328e-02 -4.47497051e-03
  -2.39828620e-02 -1.02971867e-02  4.52702902e-02 -1.05538024e-02
   2.62364205e-02 -5.94246499e-02 -7.83585832e-02  6.64470624e-03
  -7.83923045e-02 -7.26152807e-02 -6.34288117e-02  2.12332103e-02
  -7.80854300e-02  7.13305250e-02  3.76378261e-02 -8.42391104e-02
  -3.91940586e-02  1.63959116e-02  9.02236402e-02 -4.832

In [ ]:
import cohere

# Replace with your new API key
API_KEY = 'mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF'

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text or an error message.
    """
    try:
        cohere_client = cohere.Client(API_KEY)
        response = cohere_client.generate(
            model='command-xlarge-nightly',  # Changed to a generative model
            prompt=prompt,
            max_tokens=100,
            stop_sequences=["\n"]
        )
        if response and response.generations:
            return response.generations[0].text.strip()
        else:
            return "No response generated."
    except cohere.error.CohereAPIError as e: # Changed to the correct exception class
        return f"API Error: {e.message}"
    except Exception as e:
        return f"General Error: {e}"

# Sample prompt
prompt = "Explain the concept of Retrieval-Augmented Generation (RAG) in a simple manner."

# Generate response
response_text = generate_response(prompt)
print("Generated Response:", response_text)

Generated Response: Retrieval-Augmented Generation (RAG) is a fancy way of saying that a computer system can find information from a large collection of data and then use that information to generate a helpful response or answer. Imagine you have a super-smart robot friend who can quickly search through thousands of books in a library and then use what it finds to have a conversation with you.


**Build a simple frontend interface using Streamlit, allowing users to
upload PDF documents and ask questions**


In [13]:
# Install required libraries
!pip install streamlit PyPDF2 pinecone-client cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [14]:
import streamlit as st
import pinecone
import cohere
import PyPDF2
import pickle
from io import StringIO


In [17]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone using the updated approach
pc = Pinecone(api_key="bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6")

# Now check if your index exists, if not create it
index_name = 'sample-movies'

# List existing indexes to see if your index is already created
if index_name not in pc.list_indexes().names():
    # Create the index
    pc.create_index(
        name=index_name,
        dimension=1024,  # Ensure this matches your embedding dimension
        metric='cosine',  # Or use another similarity metric if needed
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Connect to your Pinecone index
index = pc.Index(index_name)


In [18]:
# Streamlit app interface
st.title("Interactive QA Bot")

# Upload PDF document
uploaded_file = st.file_uploader("Upload a PDF document", type="pdf")

if uploaded_file is not None:
    # Extract text from the uploaded PDF using PyPDF2
    pdf_reader = PyPDF2.PdfFileReader(uploaded_file)
    document_text = ""

    # Iterate through pages and extract text
    for page_num in range(pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        document_text += page.extract_text()

    # Show extracted text to the user
    st.write("Extracted Text from PDF:")
    st.write(document_text)

    # Convert document text to embeddings (Cohere)
    response = co.embed(texts=[document_text])
    document_embedding = response.embeddings[0]

    # Store the document embeddings in Pinecone
    index.upsert([("document-1", document_embedding)])
    st.write("PDF uploaded and embeddings stored!")

# Input for user query
query = st.text_input("Ask a question about the document:")

if query:
    # Embed the user's query using Cohere
    query_embedding = co.embed(texts=[query]).embeddings[0]

    # Search Pinecone for relevant document segments
    result = index.query(queries=[query_embedding], top_k=1)
    matched_id = result["matches"][0]["id"]
    matched_score = result["matches"][0]["score"]

    # Retrieve the most relevant document segment based on match score
    if matched_score > 0.7:  # A score threshold for relevance
        st.write("Relevant Document Segment:")
        st.write(document_text)  # This will need further refinement to show the correct segment
    else:
        st.write("No relevant segment found for your query.")


2024-09-18 11:38:21.055 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 11:38:21.149 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-18 11:38:21.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 11:38:21.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 11:38:21.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 11:38:21.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 11:38:21.159 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 11:38:21.160 Thread 'MainThread': mi

In [22]:
!pip install streamlit pyngrok



In [23]:
%%writefile app.py

import streamlit as st

st.title("My Interactive QA Bot")
st.write("This is an interactive QA bot for answering business-related questions.")


Writing app.py


In [26]:
# Install necessary libraries
!pip install streamlit pyngrok PyPDF2 cohere pinecone-client

# Create a temporary Streamlit app script
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import PyPDF2

st.title("Interactive QA Bot with PDF Support")
st.write("Upload a PDF and ask questions about its content.")

# File upload functionality
uploaded_file = st.file_uploader("Upload a PDF", type="pdf")

if uploaded_file is not None:
    # Extract text from PDF
    pdf_reader = PyPDF2.PdfReader(uploaded_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    st.write("Extracted text from the PDF:")
    st.write(text)
    st.write("You can now ask questions about the content.")
""")

# Run the Streamlit server in the background
from pyngrok import ngrok

# Start the Streamlit server
!streamlit run app.py &

# Expose the app on port 8501 using ngrok
public_url = ngrok.connect(port='8501')
print(f"Streamlit app URL: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.228.73:8501

  Stopping...


ERROR:pyngrok.process.ngrok:t=2024-09-18T13:20:24+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-09-18T13:20:24+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [29]:
import cohere
import pinecone

# Initialize Cohere client
co = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Initialize Pinecone client
from pinecone import Pinecone

# Create a Pinecone instance
pc = Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6')

# Connect to the Pinecone index
index = pc.Index('sample-movies')


In [33]:
!pip install pyngrok


In [32]:
import streamlit as st
import cohere
import pinecone
import numpy as np

# Initialize Cohere and Pinecone clients
co = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')
pc = Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6')
index = pc.Index('sample-movies')

def query_pinecone(query, top_k=5):
    # Encode the query using Cohere
    response = co.embed(texts=[query])
    query_embedding = response.embeddings[0]

    # Query Pinecone index
    results = index.query(query_embedding.tolist(), top_k=top_k)
    return results

def get_answer_from_cohere(context, query):
    # Construct prompt for Cohere
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"

    # Get the response from Cohere
    response = co.generate(prompt=prompt, max_tokens=150)
    return response.generations[0].text.strip()

def main():
    st.title("Interactive QA Bot")
    st.write("Upload a PDF document and ask questions.")

    # File uploader for PDF documents
    uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")

    if uploaded_file is not None:
        # Process the uploaded PDF
        # Extract text from PDF (you should implement this part)
        # Here is a placeholder
        document_text = "Extracted text from the PDF will appear here."
        st.text_area("Document Text", value=document_text, height=300)

        # Enter the query
        query = st.text_input("Ask a question:")

        if st.button("Get Answer"):
            if query:
                # Query Pinecone to get relevant document segments
                results = query_pinecone(query)

                # Process the results and get answers from Cohere
                context = " ".join([result['text'] for result in results['matches']])
                answer = get_answer_from_cohere(context, query)

                # Display the context and the answer
                st.write("### Retrieved Context:")
                st.write(context)
                st.write("### Answer:")
                st.write(answer)
            else:
                st.write("Please enter a question.")

if __name__ == "__main__":
    main()


2024-09-18 13:23:40.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 13:23:40.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 13:23:40.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 13:23:40.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 13:23:40.844 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 13:23:40.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 13:23:40.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 13:23:40.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [38]:
!pip install pyngrok --upgrade


In [39]:
!pip install streamlit --upgrade


In [51]:
pip install --upgrade pinecone-client


In [61]:
from pyngrok import ngrok
import os
import time

# Set your ngrok authtoken
ngrok.set_auth_token('2mFIg0rlL3ECRUpXdp00MwXeX6K_5La4o6k7hQSgXTm99jUAm')

# Start the Streamlit server in the background
os.system('streamlit run app.py &')

# Wait a few seconds for Streamlit to start
time.sleep(10)  # Increased wait time for Streamlit to start

# Expose the Streamlit app on port 8501 using ngrok
public_url = ngrok.connect('8501')
print(f"Streamlit app URL: {public_url}")

# Keep the script running to maintain the ngrok tunnel
import IPython
IPython.display.display()  # Keep the notebook alive


Streamlit app URL: NgrokTunnel: "https://d010-34-125-228-73.ngrok-free.app" -> "http://localhost:8501"


we got the link now drop a file and ask anything related to the file.
